In [ ]:
## Created By Akash Modak ##
## Date 25/06/2020 Time 21:05 ##
import numpy as np 
import pandas as pd
import csv
import io
from google.colab import files
# uploaded = files.upload()
# d = pd.read_csv(io.BytesIO(uploaded['combined1.csv']))

features = []
names = []
# print(data)
with open('final.csv','rt')as f:
  data = csv.reader(f)
  for row in data:
    features.append(np.array(row[0:]))
  
# print(uploaded['combined1.csv'])
# print(d)
features = np.array(features)
print(features)
df_train = pd.read_csv('final.csv')
print('train set: {0}'.format(df_train.shape))
df_train.head(10)

Saving final.csv to final.csv
[['song_id' 'tempo' 'total_beats' ... 'frame_std' 'frame_var' 'label']
 ['1' '129.1992188' '164643' ... '4.649219877' '21.61524547' '4']
 ['2' '151.9990809' '199193' ... '1.129928778' '1.276739044' '4']
 ...
 ['961' '107.6660156' '133404' ... '1.115956794' '1.245359566' '1']
 ['962' '86.1328125' '104247' ... '1.173567726' '1.377261208' '1']
 ['963' '117.4538352' '153341' ... '1.795513679' '3.223869371' '1']]
train set: (963, 56)


,song_id,tempo,total_beats,average_beats,chroma_stft_mean,chroma_stft_std,chroma_stft_var,chroma_cq_mean,chroma_cq_std,chroma_cq_var,chroma_cens_mean,chroma_cens_std,chroma_cens_var,melspectrogram_mean,melspectrogram_std,melspectrogram_var,mfcc_mean,mfcc_std,mfcc_var,mfcc_delta_mean,mfcc_delta_std,mfcc_delta_var,rmse_mean,rmse_std,rmse_var,cent_mean,cent_std,cent_var,spec_bw_mean,spec_bw_std,spec_bw_var,contrast_mean,contrast_std,contrast_var,rolloff_mean,rolloff_std,rolloff_var,poly_mean,poly_std,poly_var,tonnetz_mean,tonnetz_std,tonnetz_var,zcr_mean,zcr_std,zcr_var,harm_mean,harm_std,harm_var,perc_mean,perc_std,perc_var,frame_mean,frame_std,frame_var,label
0,1,129.199219,164643,1276.302326,0.240892,0.301678,0.091010,0.366112,0.285707,0.081629,0.226792,0.178602,0.031899,5.013715,36.131561,1305.489746,-7.652302,58.106205,3376.331055,0.009265,1.959212,3.838512,0.132475,0.064600,0.004173,1473.438015,705.663562,497961.0631,1655.685724,498.834614,248835.97250,25.016916,6.683738,44.672352,2945.661355,1442.399044,2.080515e+06,1.276907,1.537728,2.364606,-0.013783,0.104215,0.010861,0.069444,0.046261,0.002140,-4.480000e-06,0.137462,0.018896,-0.000027,0.027201,0.000740,8.467156,4.649220,21.615245,4
1,2,151.999081,199193,1293.461039,0.275192,0.300286,0.090171,0.414688,0.282245,0.079662,0.237777,0.163693,0.026795,4.973484,35.085514,1230.993286,-7.458313,45.654728,2084.354248,0.007591,2.395649,5.739136,0.136274,0.064940,0.004217,1852.490285,407.523515,166075.4154,1895.885455,312.704488,97784.09662,24.167576,7.026549,49.372389,3805.789487,919.714851,8.458754e+05,1.492162,1.765966,3.118638,-0.006023,0.092215,0.008504,0.088848,0.038953,0.001517,3.660000e-07,0.135930,0.018477,-0.000004,0.037525,0.001408,2.594830,1.129929,1.276739,4
2,3,151.999081,161883,1217.165414,0.248626,0.297679,0.088613,0.371241,0.289661,0.083904,0.216809,0.190597,0.036327,0.437058,4.863900,23.657522,-13.042703,79.185898,6270.406250,0.006632,1.680400,2.823745,0.041848,0.026085,0.000680,1144.678307,427.171290,182475.3108,1523.492834,273.218664,74648.43815,26.073470,5.826027,33.942588,2333.598389,851.338219,7.247768e+05,0.395086,0.583674,0.340675,0.022566,0.103551,0.010723,0.039310,0.033671,0.001134,-2.177326e-02,0.039642,0.001572,-0.000003,0.012040,0.000145,8.168780,3.537997,12.517425,4
3,4,161.499023,210099,1296.907407,0.281117,0.306030,0.093654,0.399010,0.281916,0.079477,0.241224,0.158569,0.025144,3.054193,16.763285,281.007690,-8.703199,54.125675,2929.588623,0.006666,2.623608,6.883318,0.108530,0.040449,0.001636,1549.812453,371.013724,137651.1831,1862.057693,344.853866,118924.18900,25.644132,9.582786,91.829782,3055.902355,913.809451,8.350477e+05,1.198203,1.431780,2.049994,0.003348,0.094996,0.009024,0.065280,0.023577,0.000556,-9.460000e-07,0.096812,0.009372,-0.000028,0.039396,0.001552,2.987247,1.572752,2.473548,4
4,5,143.554688,183554,1274.680556,0.262019,0.316322,0.100060,0.383035,0.289213,0.083644,0.226645,0.178789,0.031965,0.878722,6.509082,42.368153,-7.695001,60.273552,3632.900879,0.007846,2.389611,5.710242,0.061965,0.022247,0.000495,1557.534661,391.023980,152899.7530,1890.830789,344.951737,118991.70060,23.624107,6.285628,39.509118,3223.113844,860.966826,7.412639e+05,0.642782,0.739266,0.546513,0.007231,0.091871,0.008440,0.059199,0.024880,0.000619,-2.219704e-02,0.054460,0.002966,-0.000095,0.018283,0.000334,2.121143,1.372578,1.883970,4
5,6,107.666016,138108,1290.728972,0.246077,0.299126,0.089476,0.386659,0.275599,0.075955,0.235532,0.166907,0.027858,2.354619,14.838362,220.176971,-12.727859,66.523636,4425.394531,0.008221,2.595447,6.736344,0.094528,0.037589,0.001413,1172.522254,319.636064,102167.2137,1221.171183,272.712860,74372.30390,26.303747,8.301983,68.922920,2140.870388,763.747979,5.833110e+05,0.985097,1.185226,1.404761,-0.006452,0.096231,0.009260,0.061658,0.017812,0.000317,-5.790000e-06,0.088172,0.007774,-0.000053,0.031247,0.000976,2.662168,1.282054,1.643663,4
6,7,129.199219,157403,1345.324786,0.220004,0.312483,0.097646,0.343894,0.296317,0.087804,0.210405,0.197644,0.0

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
plt.style.use('dark_background')

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM

Using TensorFlow backend.


In [ ]:
training_set = df_train.iloc[:,1:54].values

In [ ]:
print(training_set)

[[1.29199219e+02 1.64643000e+05 1.27630233e+03 ... 7.39908000e-04
  8.46715646e+00 4.64921988e+00]
 [1.51999081e+02 1.99193000e+05 1.29346104e+03 ... 1.40814900e-03
  2.59482993e+00 1.12992878e+00]
 [1.51999081e+02 1.61883000e+05 1.21716541e+03 ... 1.44966000e-04
  8.16878005e+00 3.53799729e+00]
 ...
 [1.07666016e+02 1.33404000e+05 1.30788235e+03 ... 2.44099400e-03
  4.02285714e+00 1.11595679e+00]
 [8.61328125e+01 1.04247000e+05 1.27130488e+03 ... 3.44322000e-04
  4.29220862e+00 1.17356773e+00]
 [1.17453835e+02 1.53341000e+05 1.32190517e+03 ... 6.21482000e-04
  5.46133333e+00 1.79551368e+00]]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled)

[[0.42222222 0.42632215 0.36471016 ... 0.17761222 0.2403659  0.20791217]
 [0.5751634  0.60561918 0.38202038 ... 0.3407327  0.04347995 0.01975345]
 [0.5751634  0.41199915 0.30505108 ... 0.03238434 0.23036201 0.14850071]
 ...
 [0.27777778 0.26420754 0.39656904 ... 0.59285458 0.09135851 0.01900644]
 [0.13333333 0.11289745 0.35966859 ... 0.08104799 0.10038926 0.02208661]
 [0.34343434 0.36767049 0.41071568 ... 0.14870393 0.13958739 0.05533892]]


In [ ]:
X_train = []
y_train = []
for i in range(60, 963):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
x_val = X_train[-100:]
y_val = y_train[-100:]
X_train = X_train[:-100]
y_train = y_train[:-100]
# print(X_train)

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 64, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 64, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 64))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 64, activation='relu'))
regressor.add(Dense(64, activation='relu'))

# regressor.add(Dense(1))

regressor.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4), metrics=['accuracy'])

regressor.fit(X_train, y_train, epochs = 100, validation_data=(x_val,y_val), batch_size = 32)

Train on 803 samples, validate on 100 samples
Epoch 1/100
803/803 [==============================] - 4s 5ms/step - loss: 0.6935 - accuracy: 0.0000e+00 - val_loss: 0.6932 - val_accuracy: 0.0000e+00
Epoch 2/100
803/803 [==============================] - 3s 4ms/step - loss: 0.6932 - accuracy: 0.0125 - val_loss: 0.6931 - val_accuracy: 0.5300
Epoch 3/100
803/803 [==============================] - 3s 4ms/step - loss: 0.6932 - accuracy: 0.2042 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 4/100
803/803 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.5243 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 5/100
803/803 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.6476 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 6/100
803/803 [==============================] - 3s 4ms/step - loss: 0.6931 - accuracy: 0.7422 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 7/100
803/803 [==============================] - 3s 4ms/step - loss: 0.6931 - accu